# Generate displacement data using OPERA CSLC-S1 from ASF DAAC

---    

### Data Used in the Example:   

- **10 meter (Northing) x 5 meter (Easting) North America OPERA Coregistered Single Look Complex from Sentinel-1 products**
    - This dataset contains Level-2 OPERA coregistered single-look-complex (CSLC) data from Sentinel-1 (S1). <span style="color:red"> The data in this example are geocoded CSLC-S1 data covering the volcanoes in the Big Island, Hawaii</span>. 
    
    - The OPERA project is generating geocoded burst-wise CSLC-S1 products over North America which includes USA and US Territories within 200 km from the US border, Canada, and all mainland countries from the southern US border down to and including Panama. Each pixel within a burst SLC is represented by a complex number and contains both the amplitude and phase information. The CSLC-S1 products are distributed over projected map coordinates using the Universal Transverse Mercator (UTM) projection with spacing in the X- and Y-directions of 5 m and 10 m, respectively. Each OPERA CSLC-S1 product is distributed as a HDF5 file following the CF-1.8 convention with separate groups containing the data raster layers, the low-resolution correction layers, and relevant product metadata.

    - For more information about the OPERA project and other products please visit our website at https://www.jpl.nasa.gov/go/opera .

Please refer to the [OPERA Product Specification Document](https://d2pn8kiwq2w21t.cloudfront.net/documents/ProductSpec_CSLC_URS314199.pdf) for details about the CSLC-S1 product.

*Prepared by M. Grace Bato*

---

## 0. Setup your conda environment

Assuming you have [miniconda](https://docs.conda.io/projects/miniconda/en/latest/miniconda-install.html) or conda installed. Open your terminal and run the following:
```
> conda create -n opera_cslc --yes
> conda activate opera_cslc
> conda install -c conda-forge mamba --yes
> mamba env update -p ${CONDA_PREFIX} --file "opera_cslc_env.yml"
> pip install -U git+https://github.com/scottstanie/dolphin.git@main
```

---    

## 1. Load Python modules

In [1]:
## Load necessary modules
%load_ext watermark

import asf_search as asf
import geopandas as gpd

import numpy as np
import getpass
import folium
import datetime as dt
from shapely.geometry import box
import rioxarray
import numpy as np
from dolphin.workflows import _cli_config as dconfig
from dolphin.workflows import _cli_run as drun

%watermark --iversions

import os

numpy     : 1.26.2
dolphin   : 0.7.0.post1.dev23+gbd31392
geopandas : 0.14.1
folium    : 0.15.1
asf_search: 6.7.2
rioxarray : 0.15.0



In [2]:
## Load plotting module
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
## Load pandas and setup config
import pandas as pd
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
## Avoid lots of these warnings printing to notebook from asf_search
import warnings
warnings.filterwarnings('ignore')

## 2. Query and download OPERA CSLCs using `asf_search`

In [5]:
## Enter user-defined parameters
aoi = box(-155.88, 19.0, -155.014, 20.1962)     # (W, S, E, N)
wavelength = 0.056
orbitPass = asf.ASCENDING
pathNumber = 124
referenceDate = dt.datetime.fromisoformat('2023-12-14 00:00:00')         #'YYYY-MM-DD HH:MM:SS'
secondaryDate = dt.datetime.fromisoformat('2023-12-26 23:59:59')         #'YYYY-MM-DD HH:MM:SS'
savedir = './BigIsland'

In [6]:
## Search for OPERA CSLC data in ASF DAAC
search_params = dict(
    intersectsWith= aoi.wkt,
    dataset='OPERA-S1',
    processingLevel='CSLC',
    flightDirection = orbitPass,
    start=referenceDate,
    end=secondaryDate
)

## Return results
results = asf.search(**search_params)
len(results)

67

In [7]:
## Display the queried data
gf = gpd.GeoDataFrame.from_features(results.geojson(), crs='EPSG:4326')
gf.head()

,geometry,beamModeType,browse,bytes,centerLat,centerLon,faradayRotation,fileID,flightDirection,groupID,granuleType,insarStackId,md5sum,offNadirAngle,orbit,pathNumber,platform,pointingAngle,polarization,processingDate,processingLevel,sceneName,sensor,startTime,stopTime,url,pgeVersion,fileName,frameNumber,beamMode,additionalUrls,operaBurstID
0,"POLYGON ((-156.15680 20.12214, -155.30759 20.2...",IW,[https://datapool.asf.alaska.edu/BROWSE/OPERA-...,None,None,None,None,OPERA_L2_CSLC-S1_T124-264311-IW2_20231226T0431...,ASCENDING,S1A_IWDV_0062_0067_051821_124,None,None,None,None,51821,124,Sentinel-1A,None,[VV],2023-12-27T08:13:33Z,CSLC,OPERA_L2_CSLC-S1_T124-264311-IW2_20231226T0431...,C-SAR,2023-12-26T04:31:33Z,2023-12-26T04:31:36Z,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,2.1.0,OPERA_L2_CSLC-S1_T124-264311-IW2_20231226T0431...,None,IW,[https://datapool.asf.alaska.edu/CSLC/OPERA-S1...,T124_264311_IW2
1,"POLYGON ((-155.30681 20.16151, -154.55619 20.2...",IW,[https://datapool.asf.alaska.edu/BROWSE/OPERA-...,None,None,None,None,OPERA_L2_CSLC-S1_T124-264310-IW3_20231226T0431...,ASCENDING,S1A_IWDV_0062_0067_051821_124,None,None,None,None,51821,124,Sentinel-1A,None,[VV],2023-12-27T08:24:38Z,CSLC,OPERA_L2_CSLC-S1_T124-264310-IW3_20231226T0431...,C-SAR,2023-12-26T04:31:31Z,2023-12-26T04:31:34Z,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,2.1.0,OPERA_L2_CSLC-S1_T124-264310-IW3_20231226T0431...,None,IW,[https://datapool.asf.alaska.edu/CSLC/OPERA-S1...,T124_264310_IW3
2,"POLYGON ((-156.12276 19.95542, -155.27450 20.1...",IW,[https://datapool.asf.alaska.edu/BROWSE/OPERA-...,None,None,None,None,OPERA_L2_CSLC-S1_T124-264310-IW2_20231226T0431...,ASCENDING,S1A_IWDV_0062_0067_051821_124,None,None,None,None,51821,124,Sentinel-1A,None,[VV],2023-12-27T08:12:07Z,CSLC,OPERA_L2_CSLC-S1_T124-264310-IW2_20231226T0431...,C-SAR,2023-12-26T04:31:30Z,2023-12-26T04:31:33Z,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,2.1.0,OPERA_L2_CSLC-S1_T124-264310-IW2_20231226T0431...,None,IW,[https://datapool.asf.alaska.edu/CSLC/OPERA-S1...,T124_264310_IW2
3,"POLYGON ((-155.27364 19.99466, -154.52386 20.1...",IW,[https://datapool.asf.alaska.edu/BROWSE/OPERA-...,None,None,None,None,OPERA_L2_CSLC-S1_T124-264309-IW3_20231226T0431...,ASCENDING,S1A_IWDV_0062_0067_051821_124,None,None,None,None,51821,124,Sentinel-1A,None,[VV],2023-12-27T08:23:22Z,CSLC,OPERA_L2_CSLC-S1_T124-264309-IW3_20231226T0431...,C-SAR,2023-12-26T04:31:28Z,2023-12-26T04:31:31Z,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,2.1.0,OPERA_L2_CSLC-S1_T124-264309-IW3_20231226T0431...,None,IW,[https://datapool.asf.alaska.edu/CSLC/OPERA-S1...,T124_264309_IW3
4,"POLYGON ((-156.08868 19.78844, -155.24136 19.9...",IW,[https://datapool.asf.alaska.edu/BROWSE/OPERA-...,None,None,None,None,OPERA_L2_CSLC-S1_T124-264309-IW2_20231226T0431...,ASCENDING,S1A_IWDV_0062_0067_051821_124,None,None,None,None,51821,124,Sentinel-1A,None,[VV],2023-12-27T08:10:42Z,CSLC,OPERA_L2_CSLC-S1_T124-264309-IW2_20231226T0431...,C-SAR,2023-12-26T04:31:27Z,2023-12-26T04:31:30Z,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,2.1.0,OPERA_L2_CSLC-S1_T124-264309-IW2_20231226T0431...,None,IW,[https://datapool.asf.alaska.edu/CSLC/OPERA-S1...,T124_264309_IW2


In [8]:
## Filter data based on specified track number
gf = gf[gf.pathNumber==pathNumber]
cslc = gf[['operaBurstID','startTime', 'stopTime', 'url', 'geometry']]
cslc['startTime'] = pd.to_datetime(cslc.startTime).dt.date
cslc['stopTime'] = pd.to_datetime(cslc.stopTime).dt.date
cslc = cslc.drop_duplicates(subset=['operaBurstID', 'startTime'], ignore_index=True)
cslc

,operaBurstID,startTime,stopTime,url,geometry
0,T124_264311_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.15680 20.12214, -155.30759 20.2..."
1,T124_264310_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.30681 20.16151, -154.55619 20.2..."
2,T124_264310_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.12276 19.95542, -155.27450 20.1..."
3,T124_264309_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.27364 19.99466, -154.52386 20.1..."
4,T124_264309_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.08868 19.78844, -155.24136 19.9..."
5,T124_264308_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.23062 19.82981, -154.49158 19.9..."
6,T124_264308_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.05467 19.62171, -155.19841 19.7..."
7,T124_264307_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.19627 19.66368, -154.45935 19.7..."
8,T124_264307_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.02067 19.45498, -155.16404 19.6..."
9,T124_264306_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.16279 19.49752, -154.42713 19.6..."


In [9]:
## Choose the reference and secondary dates
ref = referenceDate.date()
sec = secondaryDate.date()

print(f'Reference Date: {ref}')
print(f'Secondary Date: {sec}')

## Drop other dates
cslc = cslc.loc[cslc.startTime.isin([ref,sec])]
cslc

Reference Date: 2023-12-14
Secondary Date: 2023-12-26


,operaBurstID,startTime,stopTime,url,geometry
0,T124_264311_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.15680 20.12214, -155.30759 20.2..."
1,T124_264310_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.30681 20.16151, -154.55619 20.2..."
2,T124_264310_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.12276 19.95542, -155.27450 20.1..."
3,T124_264309_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.27364 19.99466, -154.52386 20.1..."
4,T124_264309_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.08868 19.78844, -155.24136 19.9..."
5,T124_264308_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.23062 19.82981, -154.49158 19.9..."
6,T124_264308_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.05467 19.62171, -155.19841 19.7..."
7,T124_264307_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.19627 19.66368, -154.45935 19.7..."
8,T124_264307_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.02067 19.45498, -155.16404 19.6..."
9,T124_264306_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.16279 19.49752, -154.42713 19.6..."


In [10]:
## Ensure that you have 2 cslcs per operaBurstID
cslc.operaBurstID.value_counts()

operaBurstID
T124_264311_IW2    2
T124_264310_IW3    2
T124_264303_IW2    2
T124_264303_IW3    2
T124_264304_IW2    2
T124_264304_IW3    2
T124_264305_IW2    2
T124_264305_IW3    2
T124_264306_IW2    2
T124_264306_IW3    2
T124_264307_IW2    2
T124_264307_IW3    2
T124_264308_IW2    2
T124_264308_IW3    2
T124_264309_IW2    2
T124_264309_IW3    2
T124_264310_IW2    2
T124_264302_IW3    2
Name: count, dtype: int64

In [11]:
## Keep only bursts that have at least 2 counts of data
counts = cslc.operaBurstID.value_counts()
cslc = cslc.loc[~cslc['operaBurstID'].isin(counts[counts < 2].index)].reset_index(drop=True)
cslc

,operaBurstID,startTime,stopTime,url,geometry
0,T124_264311_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.15680 20.12214, -155.30759 20.2..."
1,T124_264310_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.30681 20.16151, -154.55619 20.2..."
2,T124_264310_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.12276 19.95542, -155.27450 20.1..."
3,T124_264309_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.27364 19.99466, -154.52386 20.1..."
4,T124_264309_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.08868 19.78844, -155.24136 19.9..."
5,T124_264308_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.23062 19.82981, -154.49158 19.9..."
6,T124_264308_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.05467 19.62171, -155.19841 19.7..."
7,T124_264307_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.19627 19.66368, -154.45935 19.7..."
8,T124_264307_IW2,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-156.02067 19.45498, -155.16404 19.6..."
9,T124_264306_IW3,2023-12-26,2023-12-26,https://datapool.asf.alaska.edu/CSLC/OPERA-S1/...,"POLYGON ((-155.16279 19.49752, -154.42713 19.6..."


In [12]:
## Basemap function for Folium
def getbasemaps():
    # Add custom base maps to folium
    basemaps = {
        'Google Satellite Hybrid': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Satellite',
            overlay = True,
            control = True,
            # opacity = 0.8,
            show = False
        ),
        'Esri Satellite': folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = True,
            control = True,
            #opacity = 0.8,
            show = False
        )
    }
    return basemaps

In [13]:
# Interactive map to visualize the boundaries of the selected OPERA CSLCs
m = folium.Map(location=[aoi.centroid.y, aoi.centroid.x], zoom_start=4, tiles="CartoDB positron")

# Add custom basemaps
basemaps = getbasemaps()
for basemap in basemaps:
    basemaps[basemap].add_to(m)

# layer Control
m.add_child(folium.LayerControl())

## RLE sites
for _, r in cslc.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "red"}, control=False)
    folium.Popup(r["operaBurstID"]).add_to(geo_j)
    geo_j.add_to(m)

m

In [14]:
## Download the relevant OPERA CSLCs to a temporary "cslc" folder 
os.makedirs(f'{savedir}/cslc', exist_ok='True')
asf.download_urls(urls=cslc.url, path=f'{savedir}/cslc', processes=10)

## 3. Create burst-wise IFGs, stitch, and unwrap using Dolphin

In [15]:
# Go to the working/save directory
os.chdir(f'{savedir}')

In [16]:
# # Create waterMask if waterMask is not available
# # You must install isce2, mintpy, and gdal
# # Run wbd.py from isce2, generate mask using from MintPy, crop mask to the same area extent of the IFG using gdalwarp

# !wbd.py 18 21 -157 -154
# !generate_mask.py swbdLat_N18_N21_Lon_W157_W154.wbd --max 0.5 -o waterMask.h5
# !save_gdal.py waterMask.h5 -o waterMask.tif
# !gdalwarp -t_srs EPSG:32605 -te 161040.0000000000000000 2075280.0000000000000000 368430.0000000000000000 2271570.0000000000000000 waterMask.tif sub_waterMask_utm.tif -ot Byte -overwrite -tr 30 -30

In [17]:
# # For this Big Island case, ensure you have the mask_file.
dconfig.create_config(outfile='dolphin_config.yaml', slc_files='./cslc/*.h5',\
                      subdataset='/data/VV', strides=[6, 3], n_workers=4, n_parallel_bursts=2, threads_per_worker= 16, mask_file='./WBD/sub_waterMask_utm.tif')

# # No unwrapping option: --no-unwrap=True
# dconfig.create_config(outfile='dolphin_config.yaml', slc_files='./cslc/*.h5',\
#                       subdataset='/data/VV', strides=[6, 3], n_workers=2, threads_per_worker=16,  mask_file='./WBD/sub_waterMask_utm.tif', no_unwrap=True)

## Via command line input
# !dolphin config --slc-files ./cslc/*.h5 --subdataset "/data/VV" --strides 6 3 --n-workers 2 --threads-per-worker 16 --mask-file './WBD/sub_waterMask_utm.tif'
# !dolphin config --slc-files ./cslc/*.h5 --subdataset "/data/VV" --strides 6 3 --n-workers 2 --threads-per-worker 16 --mask-file './WBD/sub_waterMask_utm.tif' --no-unwrap

Saving configuration to dolphin_config.yaml


In [18]:
%%time
drun.run('dolphin_config.yaml')

## Via command line input
# !dolphin run dolphin_config.yaml

[2024-01-17 16:30:13] INFO     Found SLC files from 18 bursts                                    ]8;id=531582;file:///mnt/trappist-r0/bato/tools/anaconda3/envs/calval-DISP_dev/lib/python3.11/site-packages/dolphin/workflows/displacement.py\displacement.py]8;;\:]8;id=6861;file:///mnt/trappist-r0/bato/tools/anaconda3/envs/calval-DISP_dev/lib/python3.11/site-packages/dolphin/workflows/displacement.py#76\76]8;;\

[2024-01-17 16:30:14] INFO     Running wrapped phase         ]8;id=674191;file:///mnt/trappist-r0/bato/tools/anaconda3/envs/calval-DISP_dev/lib/python3.11/site-packages/dolphin/workflows/wrapped_phase.py\wrapped_phase.py]8;;\:]8;id=617197;file:///mnt/trappist-r0/bato/tools/anaconda3/envs/calval-DISP_dev/lib/python3.11/site-packages/dolphin/workflows/wrapped_phase.py#47\47]8;;\
                               estimation in                                    
                               /u/trappist-r0/bato/work/OPER                    
                               A_Applications/CSLC/Discover/                    
                               BigIsland/t124_264302_iw3                        
[2024-01-17 16:30:14] INFO     Running wrapped phase         ]8;id=236623;file:///mnt/trappist-r0/bato/tools/anaconda3/envs/calval-DISP_dev/lib/python3.11/site-packages/dolphin/workflows/wrapped_phase.py\wrapped_phase.py]8;;\:]8;id=742282;file:///mnt/trappist-r0/bato/tools/anaconda3

## 4. Visualize the IFG and UNW data

In [ ]:
# Get reference and secondary dates
ref = referenceDate.strftime('%Y%m%d')
sec = secondaryDate.strftime('%Y%m%d')

print(f'Reference Date: {ref}')
print(f'Secondary Date: {sec}')

# Read the merged wrapped file
src = rioxarray.open_rasterio(f'./interferograms/{ref}_{sec}.int')
src = src.rio.reproject("EPSG:4326")[0]         # Reproject coordinates to EPSG:4326
minlon,minlat,maxlon,maxlat = src.rio.bounds()
merged_ifg = np.angle(src)
merged_ifg[merged_ifg==0] = np.nan              # Mask zero values
src = []


In [ ]:
# Define bounding box
bbox = [minlon,maxlon,minlat,maxlat]

# Plot using Matplotlib
fig, ax = plt.subplots(figsize=[12,9])
cax = ax.imshow(merged_ifg, cmap='jet',interpolation='nearest', origin='upper',extent=bbox)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title(f'WRP_{ref}-{sec}_{orbitPass[0]}{pathNumber}',fontsize=12)
cbar = fig.colorbar(cax,orientation='vertical',fraction=0.01,pad=0.02)
cbar.set_ticks([-np.pi, 0., np.pi])
cbar.set_ticklabels([r'$-\pi$', '$0$', r'$\pi$'])
plt.show()

In [ ]:
## Set Reference Points
xsize = 0.000277778; ysize = -0.000277778           # Assuming 30-m resolution
refx = 2642; refy = 2700;
reflon = minlon + (refx * xsize)
reflat = maxlat + (refy * ysize)

In [ ]:
# Read the unwrapped file
src = rioxarray.open_rasterio(f'./unwrapped/{ref}_{sec}.unw.tif')
merged_unw = src.rio.reproject("EPSG:4326")[0] * wavelength / (-4 * np.pi)
src = []

# Plot using Matplotlib
fig, ax = plt.subplots(figsize=[12,9])
merged_unw_cm = merged_unw*100
merged_unw_cm -= merged_unw_cm[refy, refx]              # Reference the data to a point
cax = ax.imshow(merged_unw_cm, cmap='RdBu_r',interpolation='nearest', origin='upper', extent=bbox, vmin=np.min(merged_unw_cm), vmax=-np.min(merged_unw_cm))
ax.plot(reflon, reflat, 'ks')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title(f'UNW_{ref}-{sec}_{orbitPass[0]}{pathNumber}',fontsize=12)
cbar = fig.colorbar(cax,orientation='vertical',fraction=0.01,pad=0.02)
cbar.set_label('LOS Displacement (cm)', rotation=270, labelpad=15)
plt.show()

In [ ]:
# Delete temporary "cslc" folder to save storage space
!rm -rvf ./cslc